In [10]:
# %%writefile main.py
import socket
import cv2 as cv
from cvzone.HandTrackingModule import HandDetector

cap = cv.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 640)
h, _, _ = cap.read()[1].shape

# Hand Detector
detector = HandDetector(maxHands=1, detectionCon=0.8)

# Socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # UDP
sereverAddressPort = ('127.0.0.1', 5052) # (address, port)

while True:
    success, img = cap.read()
    
    # Hands
    hands, img = detector.findHands(img)
    
    data = []
    # Landmark values
    if hands:
        # Get the first hand
        hand = hands[0]
        
        # Get the landmark list
        lmList = hand["lmList"]
        
        for lm in lmList:
            data.extend([lm[0], h - lm[1], lm[2]]) # Flip y axis to match the unity coordinate system.
            
        # Sending Data
        sock.sendto(str.encode(str(data)), sereverAddressPort)
    
    
    cv.imshow("Image", img)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv.destroyAllWindows()